In [33]:
# pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Auth at first time
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=132657856087-t2gdh3j2s3fcu1ugngr22ji2e7ksfs62.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
Modu

Authentication successful.


In [35]:
from __future__ import print_function
import json
import requests
import os
import os.path
import shutil
import errno
import sys
import urllib
import pandas as pd
from IPython.display import display

# facepy
import facepy

def gdrive_get_list():
    drive = GoogleDrive(gauth)

    # Auto-iterate through all files that matches this query
    file_list = drive.ListFile({'q': "'root' in parents"}).GetList()
    for file1 in file_list:
      print('title: %s, id: %s' % (file1['title'], file1['id']))

    # Paginate file lists by specifying number of max results
    for file_list in drive.ListFile({'maxResults': 10}):
      print('Received %s files from Files.list()' % len(file_list)) # <= 10
      for file1 in file_list:
        print('title: %s, id: %s' % (file1['title'], file1['id']))

def gdrive_upload_to_folder(fileName):
    # Create GoogleDrive instance with authenticated GoogleAuth instance.
    drive = GoogleDrive(gauth)

    # according to folder id
    f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": '1zlkUR9mitsedvx9trUjgxh54j9vua4go'}]})

    # Make sure to add the path to the file to upload below.
    f.SetContentFile('./downloads/'+fileName)
    f['title'] = fileName
    f.Upload()
    #print('upload completed')

    # Insert the permission.
    permission = f.InsertPermission({
                            'type': 'anyone',
                            'value': 'anyone',
                            'role': 'reader'})

    #print(f['alternateLink'])
    return f['alternateLink']

def download_file(url, down):
    
    fileName = url.split('/')[4].split('?')[0]
    fileName = urllib.parse.unquote(fileName)
    
    if down:
        # write file
        r = requests.get(url, stream=True)
        with open('./downloads/'+fileName, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
                    f.flush()
        doctype = fileName.split('.')[-1]
    else:
        fileName = ''
        
    return fileName

def write_context(fileName, context):
    # write context to txt
    if context:
        with open('./downloads/'+fileName+'.txt', 'w') as f:
                f.write(context)
    

def dl_fb_files(groupId, accessToken):
    
    # Facebook API
    graph = facepy.GraphAPI(accessToken)
    getString = '/v3.0/' + groupId + '/files?fields=download_link,from,message'
    pages = graph.get(getString, page=True, retry=1, limit=100)
    
    # df to save doc files
    df = pd.DataFrame(columns=['資源標題','年級','科目/單元','資源連結','doctype','內文'])
    # df to save other files
    df_packed = pd.DataFrame(columns=['資源標題','年級','科目/單元','資源連結','doctype','內文'])

    counter = 1
    #print(page)
    
    for page in pages:
        for post in page['data']:
            
            # make a tmp dir
            if os.path.isdir("./downloads"):
                shutil.rmtree('./downloads')
            os.mkdir('./downloads')

            # upload user 
            if 'from' in post:
                from_name = post['from']['name']
            else:
                confrom_nametext = ''

            # context
            if 'message' in post:
                context = post['message']
            else:
                context = ''

            # download link
            download_link = post['download_link']

            # download file
            fileName = download_file(download_link, 5)

            # Write context
            #write_context(fileName, context)

            #print('=====\n{}\n{}\n{}\n{}\n\n'.format(fileName, from_name, context, download_link))
            print(counter,'/',' : ',fileName)
            counter += 1

            fileName_lst = fileName.split('.')
            title = fileName_lst[0]
            doctype = fileName_lst[-1]

            if doctype in ['doc', 'docx', 'ppt', 'pptx', 'pdf']:# for doc files
                # upload to gdrive
                gdrive_link = gdrive_upload_to_folder(fileName)

                df = df.append({'資源標題':title,
                    '年級':'',
                    '科目/單元':'',
                    '資源連結':gdrive_link,
                    'doctype':doctype,
                    '內文':context
                    },ignore_index=True)
                df.to_csv('output.csv', index=False)
            else:# for packed files or others
                df_packed = df_packed.append({'資源標題':title,
                    '年級':'',
                    '科目/單元':'',
                    '資源連結':download_link,
                    'doctype':doctype,
                    '內文':context
                    },ignore_index=True)
                df_packed.to_csv('output_others.csv', index=False)
            
    display(df.head())
    display(df_packed.head())
    
    return df, df_packed

#group_id = '1841438095921683'
group_id = '247170855413829' # old wen

usr_token = 'EAAUQuUxvNIUBAEn7Qp5zTNfN2GAPSrpm15m1LFsd0iaT6EhKdZAlZBvN8g0mGDdLUHfyFm15YfIwHpxi4VAkuWvVbU2rFZAKIEamZClYe4qgeCwKqPQlTL7NSJXG0dB7NO8XjygxKw55XyF5CfFqcOHAXb84IEAc5hNVUWNeDAZDZD'
df, df_packed = dl_fb_files(group_id, usr_token)


1 /  :  心_日記.docx
2 /  :  南一六上第二課鷸蚌相爭教學.doc
3 /  :  Character.pdf
4 /  :  數學日記.rar
5 /  :  履歷自傳小書小檔案--接新班_請孩子介紹自己給你吧!.doc
6 /  :  康軒二上(一~四課).rar
7 /  :  二上第01課狗蟻備課.doc
8 /  :  一上第01課來去讀冊備課.doc
9 /  :  六上第01課叫出我的名臺灣備課.doc
10 /  :  溫老師彭老師金門分享筆記.docx
11 /  :  段老師8.21研習筆記.docx
12 /  :  圓面積推演.pptx
13 /  :  1-100的質數.pptx
14 /  :  (康軒版)心智圖一到六只有PDF檔.rar
15 /  :  104(一)135年級預習單word及PDF檔(康軒版).rar
16 /  :  104(一)一到六年級課文word檔(康軒版).rar
17 /  :  預習作業+說明.docx
18 /  :  2上康軒課文預測單.docx
19 /  :  人格特质卡简体更新.pdf
20 /  :  理解篇章的解构分析方式.pdf
21 /  :  康軒六上第一課朱子治家格言選 回饋單.doc
22 /  :  104五上翰林版1-5課資料.rar
23 /  :  溫美玉老師圓夢繪本3.rar
24 /  :  溫美玉老師圓夢繪本2.rar
25 /  :  溫美玉老師圓夢繪本1.rar
26 /  :  段淑如老師研習紀錄 104.docx
27 /  :  104五上翰林第五課棉花上的沉睡者.doc
28 /  :  104五上翰林第四課黑白間的光彩.doc
29 /  :  104五上翰林第三課一池子的綠.doc
30 /  :  104五上翰林第二課湖邊散步.doc
31 /  :  104五上翰林第一課貝殼砂.doc
32 /  :  104五上翰林文本分析.doc
33 /  :  翰林四上說明文.docx
34 /  :  翰林五上1-7課預習單結構單.rar
35 /  :  翰林六上預習單結構單.rar
36 /  :  翰林三上課文分析.rar
37 /  :  20150813六甲上學期預習A本.docx
38 /  :  康軒國語三上預習單1 (

316 /  :  第六課 鍋子裡的小白花  複習單.pdf
317 /  :  翰林第十四課身心手腦合一講義.docx
318 /  :  從國語課本深入閱讀.pptx
319 /  :  國語課文14 - 延伸寫作.docx
320 /  :  一上康軒第七課爬山.doc
321 /  :  14小樹.docx
322 /  :  第五課   我不怕複習單.pdf
323 /  :  文字變變變.docx
324 /  :  09沉思三帖.pptx
325 /  :  第四課  玩玩具複習單.pdf
326 /  :  13筆記四則.docx
327 /  :  國語課文13 -筆記四則-延伸寫作-印7張.docx
328 /  :  第十二課圓夢之旅.docx
329 /  :  第十四課小樹.doc
330 /  :  原住民委員會-原來如此.pdf
331 /  :  第三課   吹泡泡複習單.pdf
332 /  :  恐龍滅絕.docx
333 /  :  第十三課筆記四則.doc
334 /  :  國語課文13 -筆記四則-預習單.doc
335 /  :  第二課 兩人三腳複習單.pdf
336 /  :  翰林四上第十課落山風延伸寫作教案.docx
337 /  :  國語課文12 -我不是現在的我-延伸寫作.docx
338 /  :  12我不是現在的我.docx
339 /  :  六上康軒-第12課- 我願.docx
340 /  :  小六上 5-2.doc
341 /  :  國語課文11 -詩二首-延伸寫作.docx
342 /  :  平行四邊形及三角形畫高口訣.doc
343 /  :  作文批閱我最行.doc
344 /  :  考前一週讀書計畫.doc
345 /  :  中正大學磨課師課程推廣與翻轉教學導入研習會.pdf
346 /  :  三1-1 1-2空氣的特性.doc
347 /  :  寒假作業.rar
348 /  :  翰林四上四則運算.pdf
349 /  :  風車製作.ppt
350 /  :  103學年度六年級上學期第二次國語閱讀理解試卷.docx
351 /  :  翰林_六上_U5-U7數學解題思維試卷.docx
352 /  :  票選2014代表字.doc
353 /  :  第十二課我不

617 /  :  第三課 永遠的譚爸爸-預習單.docx
618 /  :  翰林四下-黑面琵鷺.docx
619 /  :  第二課 勇敢的小巨人-預習單.docx
620 /  :  預習單_作業單4.pdf
621 /  :  十點不一樣 - __Google 塗鴉成招牌__ (2011-04-22, TVBS新聞台).mpeg.mp4
622 /  :  引人注目的GOOGLE標誌.pptx
623 /  :  評量.pdf
624 /  :  作業單.pdf
625 /  :  _親子天下_林良爺爺最愛的經典故事_金銀島.mp4
626 /  :  從從容容  穩穩當當.pptx
627 /  :  作業單.pdf
628 /  :  -薛岳 - 如果還有明天.mp4
629 /  :  努力請從今日始.pptx
630 /  :  紐伯瑞大獎整理.doc
631 /  :  國語第2課課文分析表.docx
632 /  :  我們5年級全班寫小說.pdf
633 /  :  102四下國語課文內容.docx
634 /  :  1月20_1月23日擁抱麗雲老師__備課趴版 (回覆內容).xlsx
635 /  :  南一四上L8遊子吟小書教學.doc
636 /  :  完璧歸趙由課文長作文.doc
637 /  :  獎狀.rar
638 /  :  神秘的入口通往哪裡.doc
639 /  :  102年寒假作業.rar
640 /  :  寒暑假作業封面+遊記+年菜.rar
641 /  :  1314跨年祝福.docx
642 /  :  2013~2014  新年快樂2.doc
643 /  :  四上期末比較表格.docx
644 /  :  愛的種子大集合.doc
645 /  :  夢想中的家寫作教學引導單1230.docx
646 /  :  See Your Power 看見你的影響力_竹中國小_林佳諺.pdf
647 /  :  小樹兒.docx
648 /  :  ginger1.pdf
649 /  :  taiwan.pdf
650 /  :  乘法拼圖1.docx
651 /  :  給老師的懺悔信2.pdf
652 /  :  self.pdf
653 /  :  1021009西遊記ch11-13閱讀任務單.doc
6

,資源標題,年級,科目/單元,資源連結,doctype,內文
0,心_日記,,,https://drive.google.com/a/junyiacademy.org/fi...,docx,各位老師大家好，之前小弟分享了海外版的心情小語實施過程，與孩子的作品分享後，當天有老師詢問，...
1,南一六上第二課鷸蚌相爭教學,,,https://drive.google.com/a/junyiacademy.org/fi...,doc,南一六上第二課鷸蚌相爭備課\n-------------------------------...
2,Character,,,https://drive.google.com/a/junyiacademy.org/fi...,pdf,各位老師好！今天是新加坡國慶，我又來分享啦！附上簡體字版加漢語拼音及英文註解的自製“性格卡”...
3,履歷自傳小書小檔案--接新班_請孩子介紹自己給你吧!,,,https://drive.google.com/a/junyiacademy.org/fi...,doc,【履歷自傳】小檔案創作參考計畫表
4,二上第01課狗蟻備課,,,https://drive.google.com/a/junyiacademy.org/fi...,doc,真平二上閩南語第一課『狗蟻』\n狗蟻狗蟻真無閒 來來去去佇路頂 為著照顧大家庭 規日做工做無...


,資源標題,年級,科目/單元,資源連結,doctype,內文
0,數學日記,,,https://lookaside.fbsbx.com/file/%E6%95%B8%E5%...,rar,這是這兩年設計的數學日記任務單，搭配南一版五六年級課程，仍有許多不足之處，歡迎夥伴們和我討論...
1,康軒二上(一~四課),,,https://lookaside.fbsbx.com/file/%E5%BA%B7%E8%...,rar,康軒。國語。二上。學習單與檔案\n自行設計的學習單使用方式已兩年多，大多每一課會設計不同的東...
2,(康軒版)心智圖一到六只有PDF檔,,,https://lookaside.fbsbx.com/file/%28%E5%BA%B7%...,rar,"各位老師好\n 今日最後一份資料分享\n PDF檔康軒版心智圖,..."
3,104(一)135年級預習單word及PDF檔(康軒版),,,https://lookaside.fbsbx.com/file/104%28%E4%B8%...,rar,"各位老師好\n 嘉音另外整理了一份1,3,5年級的預習單有WORD及PDF檔\n 這是較..."
4,104(一)一到六年級課文word檔(康軒版),,,https://lookaside.fbsbx.com/file/104%28%E4%B8%...,rar,各位老師好\n嘉音傳了一份康軒版的刻紋資料給大家\n希望各位老師在製作備課資料時能輕鬆一點


In [16]:
#https://lookaside.fbsbx.com/file/2-2.ppt?token=AWyebSAPL2lcDrxpSk_YYY2dmaHJPXKq2GDOiR520mkkXsIepoCjA3yvGW0KvfuF22qH1y9hVXrkGt2nMVdnJ8sn4OZgwTb5fyM4q42zmB90kmWvNJfnVB6Yrb3ZEI0u0Ux-InhBYZS7mURf6G3DHoozlpqkweXqKfavHonqCHfnok5YA6wYnLwKY0mFOmKzVcw
#https://lookaside.fbsbx.com/file/1%E6%95%B8%E5%AD%B8%E8%A7%A3%E9%A1%8C%E9%96%B1%E8%AE%80%E5%AD%B8%E7%BF%92%E6%B4%BB%E5%8B%95%E8%A8%AD%E8%A8%88_%E5%8D%97%E9%96%80%20%20%E6%9B%BE%E6%98%8E%E5%BE%B7_%E5%9C%8B%E6%96%87%E7%9A%84%E6%A3%8B%E7%9B%A4.docx?token=AWygxtrzMs5jC1AobwBoSqtgAG1pmiKjwS6IT3yU5OOLq9QghHAi99fGUqKBoW6qge6-zDfIQwiytszMsoHGPWBso0dceDRjFW7JsnCRZ0NLbjTXszgc43yXgZ9kAsksdW_Qi6k9-UnNtZBxSwViFn-MWNRq6iTHoBZMv5yMdeX-77jDRT9j47LZWrn6Gfo9N6E

NameError: name 'df' is not defined

In [53]:
df2 = pd.read_csv('output.csv')
df2.head()

,資源標題,年級,科目/單元,資源連結,doctype,內文
0,2-2,NaN,NaN,https://drive.google.com/a/junyiacademy.org/fi...,ppt,這是內文這是內文這是內文這是內文這是內文這是內文這是內文\n123123123\nddddd
1,1數學解題閱讀學習活動設計_南門 曾明德_國文的棋盤,NaN,NaN,https://drive.google.com/a/junyiacademy.org/fi...,docx,NaN
